# Lexical and Syntax Analysis

## Introduction

- Language implementation systems must analyze source code, regardless of the specific implementation approach
- Nearly all syntax analysis is based on a formal description of the syntax of the source language (BNF)

## Syntax Analysis

The **syntax analysis** portion of a language processor nearly always consists of two parts:

- A low-level part called a **lexical analyzer**
    - a finite automaton based on a regular grammar
- A high-level part called a **syntax analyzer**, or **parser**
    - a push-down automaton based on a context-free grammar, or BNF

## Lexical Analysis

- A **lexical analyzer** is a pattern matcher for character strings
- A lexical analyzer is a "front-end" for the parser
- Identifies substrings of the source program that belong together - **lexemes**
    - Lexemes match a character pattern, which is associated with a lexical category called a **token**
    - `result` is a lexeme; its token may be IDENT

Consider the following example of an assignment statement:

    result = oldsum – value / 100;

Following are the tokens and lexemes of this statement:

| Token  | Lexeme |
|--------|--------|
|IDENT| result |
|ASSIGN_OP| = |
|IDENT| oldsum |
|SUB_OP| - |
|IDENT| value |
|DIV_OP| / |
|INT_LIT| 100 |
|SEMICOLON|`;`|

## Lexical Analysis (cont.)

The lexical analyzer is usually a function that is called by the **parser** when it needs the next token.

Three approaches to building a lexical analyzer:
- Write a formal description of the tokens and use a software tool that constructs a table-driven lexical analyzer from such a description
- Design a state diagram that describes the tokens and write a program that implements the state diagram
- Design a state diagram that describes the tokens and hand-construct a table-driven implementation of the state diagram

## State Diagram

A naïve state diagram would have a transition from every state on every character in the source language - such a diagram would be very large!

![](img/state-daig.png)

## Reserved words

- Reserved words and identifiers can be recognized together
    - rather than having a part of the diagram for each reserved word
- Use a table lookup to determine whether a possible identifier is in fact a reserved word

## Example

See [Sentence Lexical Analyzer](Sentence%20Lexer.ipynb)

## The Parsing Problem

Goals of the parser, given an input program:
- Find all syntax errors; for each, produce an appropriate diagnostic message and recover quickly
- Produce the parse tree, or at least a trace of the parse tree, for the program

Two categories of parsers
- **Top down** - produce the parse tree, beginning at the root
    - Order is that of a leftmost derivation
    - Traces or builds the parse tree in preorder
- **Bottom up** - produce the parse tree, beginning at the leaves
    - Order is that of the reverse of a rightmost derivation
    
Useful parsers look only one token ahead in the input

# Top-down Parsers

## Top-down Parsers

Given a sentential form, $xA\alpha$:
- the parser must choose the correct $A$-rule to get the next sentential form in the leftmost derivation, using only the first token produced by $A$

The most common top-down parsing algorithms:
- **Recursive descent** - a coded implementation
- **LL** parsers - table driven implementation

## Recursive Descent Parsing

- There is a subprogram for each nonterminal in the grammar, which can parse sentences that can be generated by that nonterminal
- EBNF is ideally suited for being the basis for a recursive-descent parser, because EBNF  minimizes the number of nonterminals

The coding process when there is only one RHS:
- For each terminal symbol in the RHS, compare it with the next input token; if they match, continue, else there is an error
- For each nonterminal symbol in the RHS, call its associated parsing subprogram

A nonterminal that has more than one RHS requires an initial process to determine which RHS it is to parse
- The correct RHS is chosen on the basis of the next token of input (the lookahead)
- The next token is compared with the first token that can be generated by each RHS until a match is found
- If no match is found, it is a syntax error

## Building A Recursive Descent Parser

$$ S \to F \,| \, ( S + F ) $$
$$ F \to a$$

Start with a procedure `Match`, that matches the current input token against a predicted token:

In [ ]:
#load "grammartools.fsx"
open CSCI374.Lexer
open CSCI374.ParserTypes

In [12]:
let Match term (curtkn, input) = 
    if term = curtkn then
        printfn "Rule matched!!!"; (token input)
    else
        failwith (sprintf "Cannot match symbol `%A` with `%A`" term curtkn)

In [13]:
let (tkn, input) = token (Seq.toList "a+a")
tkn, input

Item1,Item2
a,"[ +, a ]"


In [14]:
let (tkn, input) = Match A (tkn, input)
tkn, input

Rule matched!!!


Item1,Item2
+,[ a ]


In [15]:
Match A (tkn, input)

Unhandled exception: System.Exception: Cannot match symbol `A` with `PLUS`
   at FSI_0017.Match[a](a term, a curtkn, FSharpList`1 input)
   at <StartupCode$FSI_0020>.$FSI_0020.main@()

## Building A Recursive Descent Parser (cont.)

To build a parsing procedure for a non-terminal **A**, we look at all productions with **A** on the lefthand side:

$$ A \to X_1 X_2 \ldots X_n \,| \, Y_1 Y_2 \ldots Y_m | \ldots$$

We match a production's righthand side by
- calling `Match` to match **terminals**, and
- calling parsing procedures to match **nonterminals**

## Building A Recursive Descent Parser (cont.)

The general form of a parsing procedure is following
```fsharp
let rec A curtkn = 
    if curtkn in Predict(X1 X2 ... Xn) then
        if X1 is Terminal then (Match X1 curtkn) else (X1 curtkn)
        if X2 is Terminal then (Match X2 curtkn) else (X2 curtkn)
        ...
    elif ... // process Y1 ... Ym
        // Handle other A →... productions
    else // No production predicted
        failwith "Syntax error"
        
```

## Example

A parsing procedure for a non-terminal $S$ in $ S \to F \,| \, ( S + F ) $

In [ ]:
let rec S tkn =
    if tkn = LPAR then
        next_tkn |> S |> Match PLUS |> F |> Match RPAR
    else
        tkn |> F

## LL(1) Grammar

LL(1) grammars are ideally suited for top-down parsing because it is always possible to correctly predict the expansion of any nonterminal.

$$\begin{equation}\begin{split}
Predict(A \to X_1 \ldots X_n ) = \textbf{if } & X_1 \ldots X_n \Rightarrow_* \varepsilon \\
\textbf{then } & FIRST(X_1 \ldots X_n) \cup FOLLOW(A) \\
\textbf{else } & FIRST(X_1 \ldots X_n)
\end{split}\end{equation}$$

**LL(1)** grammars are easy to parse in a top-down manner since predictions are always correct
- **LL(1)** grammar is always pairwise disjoint (see below)

## FIRST set

**Definition:** $FIRST(\alpha) = \{a | \alpha \Rightarrow_* a\beta \}$
- If $\alpha \Rightarrow_* \varepsilon$, $\varepsilon$ is in $FIRST(\alpha)$
- where $\Rightarrow_*$ means 0 or more derivation steps

## Notation

- Metasymbol $\Rightarrow_*$ means *derives after **zero or more** derivation steps*
    - $ S \Rightarrow_* a + a $ or $ S \Rightarrow_* S $
- Metasymbol $\Rightarrow_+$ *means **one or more** derivation steps*
    - $ F \Rightarrow_+ a$

## Example

See [Recursive Descent Parser](parsers/rd1-parser.fsx)

## Left Recursion Problem

The Left Recursion Problem
- If a grammar has **left** recursion, either direct or indirect, it cannot be the basis for a top-down parser

A grammar can be modified to remove direct left recursion as follows:
1. For each nonterminal, $A$, 
    - Group the $A$-rules as $A \rightarrow A\alpha_1 | \ldots | A \alpha_m | \beta_1 | \beta_2 | \ldots | \beta_n$ where none of the $\beta$'s begins with $A$
2. Replace the original $A$-rules with

$$ A \rightarrow \beta_1 A' | \beta_2 A' | \ldots | \beta_n A' $$
$$ A' \rightarrow \alpha_1 A' | \alpha_2 A' | \ldots | \alpha_m A' | \varepsilon $$

Symbol $\varepsilon$ specifies the empty string.
- A rule that has $\varepsilon$ as its RHS is called an **erasure rule**
  - because its use in a derivation effectively erases its LHS from the sentential form.

## Example

Grammar:

$$E  \rightarrow  E + T | T$$
$$T  \rightarrow  T * F | F$$
$$F  \rightarrow  (E) | id$$

Step 1:

For the $E$-rules, we have $\alpha_1 = +T$ and $\beta = T$, so we replace the $E$-rules with

$$E  \rightarrow  T E'$$
$$E' \rightarrow  +T E' | \varepsilon$$


## Example

Grammar:

$$E  \rightarrow  E + T | T$$
$$T  \rightarrow  T * F | F$$
$$F  \rightarrow  (E) | id$$

Step 2:

For the $T$-rules, we have $\alpha_1 = *F$ and $\beta = F$, so we replace the $T$-rules with

$$T \rightarrow  F T'$$
$$T' \rightarrow  *F T' | \varepsilon$$

## Example

Grammar:

$$E  \rightarrow  E + T | T$$
$$T  \rightarrow  T * F | F$$
$$F  \rightarrow  (E) | id$$

Step3: 
Because there is no left recursion in the $F$-rules, they remain the same, so the complete replacement grammar is

$$E  \rightarrow  T E'$$
$$E' \rightarrow  +T E' | \varepsilon$$
$$T \rightarrow  F T'$$
$$T' \rightarrow  *F T' | \varepsilon$$
$$F  \rightarrow  (E) | id$$

This grammar generates the same language as the original grammar but is **not** left recursive.

## Pairwise Disjointness

The other characteristic of grammars that disallows top-down parsing is the lack of **pairwise disjointness**
- The inability to determine the correct RHS on the basis of the next token of input, using only the first token generated by the leftmost nonterminal in the current sentential form

## Pairwise Disjointness Test

For each nonterminal, $A$, in the grammar that has more than one RHS, for each pair of rules, $A \rightarrow \alpha_i$ and $A \rightarrow \alpha_j$, it must be true that 

$$FIRST(\alpha_i) \cap FIRST(\alpha_j) = \emptyset$$


## FIRST Construction

Rules for constructing FIRST sets:
1. If $x$ is a terminal **then** FIRST($x$) is just $\{x\}$! 
2. If there is a production $X \to \varepsilon$ **then** add $\varepsilon$ to FIRST($X$)
3. If there is a production $X \to Y_1 Y_2\ldots Y_k$ **then** add FIRST($Y_1 Y_2\ldots Y_k$) to FIRST($X$)
4. If FIRST($Y_1 Y_2\ldots Y_k$) is **either**
    1. FIRST($Y_1$) (if FIRST($Y_1$) doesn't contain $\varepsilon$)
    2. **or** (if FIRST($Y_1$) does contain $\varepsilon$) then FIRST($Y_1 Y_2\ldots Y_k$) is everything in FIRST($Y_1$) (except for $\varepsilon$) as well as everything in FIRST($Y_2\ldots Y_k$)
    3. If FIRST($Y_1$)FIRST($Y_2$)...FIRST($Y_k$) all contain $\varepsilon$ **then** add $\varepsilon$ to FIRST($Y_1 Y_2\ldots Y_k$) as well.

## Example

Grammar:

$$ A \rightarrow aB | bAb | Bb $$
$$ B \rightarrow cB | d $$

Steps:

- The $FIRST$ sets for the RHSs of the $A$-rules are
    - FIRST(aB) = {a}
    - FIRST(bAb) = {b}
    - FIRST(Bb) = FIRST(cBb) | FIRST(db) = {c ,d}
    
The FIRST sets for the RHSs of the $A$-rules are {a}, {b}, and {c, d}, which are clearly disjoint.
- passed the pairwise disjointness test

## Example

Grammar:

$$ A \rightarrow a | bB | aAc $$
$$ B \rightarrow a | aB $$

Steps:

- The $FIRST$ sets for the RHSs of the $A$-rules are
    - FIRST(a) = {a}
    - FIRST(bB) = {b}
    - FIRST(aAc) = {a}

They are intersected, thus the rule fails the test.

## Example

Grammar:

$$ A \rightarrow aB | BAb $$
$$ B \rightarrow aB | b $$

Steps:

- The $FIRST$ sets for the RHSs of the $A$-rules are
    - FIRST(aB) = {a}
    - FIRST(BAb) = FIRST(aBAb) | FIRST(bAb) = {a, b}

They are intersected, thus the rule fails the test.

## Left Factoring

In many cases, a grammar that fails the pairwise disjointness test can be modified so that it will pass the test.

Consider the rule:

$$A \rightarrow a | a[B]$$

Rule clearly does not pass the pairwise disjointness test, because both RHSs begin with the same terminal, **a**.

Replace $A \rightarrow a | a[B]$ with

$$A \rightarrow a C$$
$$C  \rightarrow \varepsilon | [B] $$

## LL Parsers

The `LL(k)` parser is a [deterministic pushdown automaton](https://en.wikipedia.org/wiki/Deterministic_pushdown_automaton) with the ability to peek on the next `k` input symbols without reading.

The stack alphabet $\Gamma = N \cup \Sigma$ where: 

- $N$ is the set of non-terminals
- $\Sigma$ the set of terminal (input) symbols with a special end-of-input (EOI) symbol `$`.

The parser stack initially contains the starting symbol above the EOI: $[\ S\ \$\ ]$

## LL Parsing

During operation, the parser repeatedly replaces the symbol $X$ on top of the stack:

- with some $\alpha$ , if $X \in N$ and there is a rule $X \to \alpha$
- with $\epsilon$, i.e. $X$ is popped off the stack, if $X \in \Sigma$.
  - In this case, an input symbol $x$ is read and if $x \neq X$, the parser rejects the input.

If the last symbol to be removed from the stack is the EOI, the parsing is successful; the automaton accepts via an empty stack.

## FOLLOW Set

**Definition:** For a nonterminal $A$ in a sentential form, say $\omega_1 A \omega_2$ where $\omega_1$ and $\omega_2$ are some string of terminals and nonterminals,

$$FOLLOW(A) = FIRST(\omega_2)$$

That is, $FOLLOW(A)$ is the set of terminals that can appear to the right of $A$ in a sentential form.

- If $B \to \omega_1 A \omega_2$ is a production and $FIRST(\omega_2)$ contains $\varepsilon$, then 

$$ FOLLOW(A) = \{ FIRST(\omega_2) - \varepsilon \} \cup FOLLOW(B)$$ 

## FOLLOW Construction

Rules for constructing FOLLOW sets:
1. First put \\$ (the end of input marker) in FOLLOW($S$) ($S$ is the start symbol)
2. If there is a production $A \to aBb$, (where $a$ can be a whole string) then everything in FIRST($b$) except for $\varepsilon$ is in FOLLOW($B$).
3. If there is a production $A \to aB$, then everything in FOLLOW($A$) is in FOLLOW($B$)
4. If there is a production $A \to aBb$, where FIRST($b$) contains $\varepsilon$, then everything in FOLLOW($A$) is in FOLLOW($B$)

## Example

Grammar:

$$E  \rightarrow  T E'$$
$$E' \rightarrow  +T E' | \varepsilon$$
$$T \rightarrow  F T'$$
$$T' \rightarrow  *F T' | \varepsilon$$
$$F  \rightarrow  (E) | id$$

Step 1: 
- FIRST: We apply rule 2 to $T' \to  \varepsilon$ and $E' \to \varepsilon$
- FOLLOW: Add \$ to the start Symbol E

|X|FIRST(X)|FOLLOW(X)|
|-|--------|---------|
|E|||$\{ \$ \}$
|E'|$\{\varepsilon\}$||
|T|||
|T'|$\{\varepsilon\}$||
|F| ||



## Example

Step 2: 
- FIRST: We apply rule 3 to $E' \to +TE'$, thus we can add everything in FIRST($+TE'$) into FIRST($E'$)
    - Since FIRST($+$) useing rule 1 is $+$, we can add $+$ to FIRST($E'$)
- FOLLOW: we apply rule 2 to $E' \to +TE'$, thus everything in FIRST($E'$) except for $\varepsilon$ should be in FOLLOW($T$)

|X |FIRST(X)|FOLLOW(X)|
|--|--------|---------|
|E ||$\{\$\}$
|E'|$\{+,\varepsilon\}$||
|T ||$\{+\}$|
|T'|$\{\varepsilon\}$||
|F ||

## Example

Step 3: 
- FIRST: We apply rule 3 to $T' \to *FT'$, thus we can add everything in FIRST($*FT'$) into FIRST($T'$)
    - Since FIRST($*$) useing rule 1 is $*$, we can add $*$ to FIRST($T'$)
- FOLLOW: We apply rule 3 to $E \to TE'$, thus we should add everything in FOLLOW($E$) into FOLLOW($E'$)

|X |FIRST(X)|FOLLOW(X)|
|--|--------|---------|
|E ||$\{\$\}$|
|E'|$\{+,\varepsilon\}$|$\{\$\}$|
|T ||$\{+\}$|
|T'|$\{*,\varepsilon\}$||
|F ||

## Example

Step 4: 
- FIRST: We apply rule 3 to two more productions: $F \to (E)$ and $F \to id$
- FOLLOW: We apply rule 2 to $T \to FT'$, thus we should add everything in FOLLOW($T$) into FOLLOW($T'$)

|X |FIRST(X)|FOLLOW(X)|
|--|--------|---------|
|E ||$\{\$\}$|
|E'|$\{+,\varepsilon\}$|$\{\$\}$|
|T ||$\{+\}$|
|T'|$\{*,\varepsilon\}$|$\{+\}$|
|F |$\{(, id\}$|

## Example

Step 5: 
- FIRST: We apply rule 3 to $T \to FT'$, thus we can add FIRST($FT'$) to FIRST($T$)
- FOLLOW: We apply rule 2 to $T' \to *FT'$, thus we should add everything in FIRST($T'$) exept for $\varepsilon$ into FOLLOW($F$)

|X |FIRST(X)|FOLLOW(X)|
|--|--------|---------|
|E ||$\{\$\}$|
|E'|$\{+,\varepsilon\}$|$\{\$\}$|
|T |$\{(, id\}$|$\{+\}$|
|T'|$\{*,\varepsilon\}$|$\{+\}$|
|F |$\{(, id\}$|$\{*\}$

## Example

Step 6: 
- FIRST: We apply rule 3 to $E \to TE'$, thus we can add FIRST($TE'$) to FIRST($E$)
- FOLLOW: We apply rule 2 to $F \to (E)$, thus everything in FIRST($)$) should be in FOLLOW($E$)

|X |FIRST(X)|FOLLOW(X)|
|--|--------|---------|
|E |$\{(, id\}$|$\{\$,)\}$|
|E'|$\{+,\varepsilon\}$|$\{\$\}$|
|T |$\{(, id\}$|$\{+\}$|
|T'|$\{*,\varepsilon\}$|$\{+\}$|
|F |$\{(, id\}$|$\{*\}$

## Example

Step 7: 
- FIRST: Finished.
- FOLLOW: We apply rule 3 to $E \to TE'$, thus everything in FOLLOW($E$) should be in FOLLOW($E'$)

|X |FIRST(X)|FOLLOW(X)|
|--|--------|---------|
|E |$\{(, id\}$|$\{\$,)\}$|
|E'|$\{+,\varepsilon\}$|$\{\$,)\}$|
|T |$\{(, id\}$|$\{+\}$|
|T'|$\{*,\varepsilon\}$|$\{+\}$|
|F |$\{(, id\}$|$\{*\}$

## Example

Step 8: 
- FIRST: Finished.
- FOLLOW: We apply rule 4 to $E' \to +TE'$, thus add everything in FOLLOW($E'$) into FOLLOW($T$) because FIRST($E'$) has $\varepsilon$

|X |FIRST(X)|FOLLOW(X)|
|--|--------|---------|
|E |$\{(, id\}$|$\{\$,)\}$|
|E'|$\{+,\varepsilon\}$|$\{\$,)\}$|
|T |$\{(, id\}$|$\{+,\$,)\}$|
|T'|$\{*,\varepsilon\}$|$\{+\}$|
|F |$\{(, id\}$|$\{*\}$

## Example

Step 9: 
- FIRST: Finished.
- FOLLOW: We apply rule 3 to $T \to FT'$, thus add everything in FOLLOW($T$) into FOLLOW($T'$)

|X |FIRST(X)|FOLLOW(X)|
|--|--------|---------|
|E |$\{(, id\}$|$\{\$,)\}$|
|E'|$\{+,\varepsilon\}$|$\{\$,)\}$|
|T |$\{(, id\}$|$\{+,\$,)\}$|
|T'|$\{*,\varepsilon\}$|$\{+,\$,)\}$|
|F |$\{(, id\}$|$\{*\}$

## Example

Step 10: 
- FIRST: Finished.
- FOLLOW: We apply rule 4 to $T' \to *FT'$, thus add everything in FOLLOW($T'$) into FOLLOW($F$)

|X |FIRST(X)|FOLLOW(X)|
|--|--------|---------|
|E |$\{(, id\}$|$\{\$,)\}$|
|E'|$\{+,\varepsilon\}$|$\{\$,)\}$|
|T |$\{(, id\}$|$\{+,\$,)\}$|
|T'|$\{*,\varepsilon\}$|$\{+,\$,)\}$|
|F |$\{(, id\}$|$\{*,+,\$,)\}$

## LL(1) Parsing Table

LL(1) parser uses provided (generated) parsing table. The table provides the following mapping:

- **row**: top-of-stack symbol $X$
- **column**: $|w| \le k$ lookahead buffer contents
- **cell**: rule number for $X \to \alpha$ or $\varepsilon$

If the parser cannot perform a valid transition, the input is rejected (empty cells).
- To make the table more compact, only the non-terminal rows are commonly displayed, since the action is the same for terminals. 

## LL(1) Parsing Table Contruction

Constructing LL(1) parsing tables is relatively easy. The table is constructed using the following algorithm.


For every rule $A \to \alpha$ in the grammar:

1. If $\alpha$ can derive a string starting with $a$ (i.e., for all $a \in FIRST(\alpha)$),

$$Table[A, a] = A \to \alpha$$

2. If $\alpha$ can derive the empty symbol, $\varepsilon$, then, for all $b$ that can follow a string derived from $A$ (i.e., for all $b \in FOLLOW(A)$),

$$Table[A, b] = A \to \alpha $$

## Example

For the following the non-left-recursive, left-factored expression grammar construct LL(1) parsing table:

$$E \to T E'$$
$$E'\to +T E' | \varepsilon$$
$$T \to F T'$$
$$T'\to *F T' | \varepsilon$$
$$F \to (E) | id$$

Step 1:

Consider the production $E \to T E'$. Since

$$FIRST(TE') = \{ (, id \}$$

Then
    
$$Table[E,(] = E \to TE'$$ 
$$Table[E,id] = E \to TE'$$

## Example

For the following the non-left-recursive, left-factored expression grammar construct LL(1) parsing table:

$$E \to T E'$$
$$E'\to +T E' | \varepsilon$$
$$T \to F T'$$
$$T'\to *F T' | \varepsilon$$
$$F \to (E) | id$$

Step 2:

Consider the production $E'\to +T E'$. Since

$$FIRST(+T E') = \{ + \}$$

Then
    
$$Table[E',+] = E'\to +T E'$$ 

## Example

For the following the non-left-recursive, left-factored expression grammar construct LL(1) parsing table:

$$E \to T E'$$
$$E'\to +T E' | \varepsilon$$
$$T \to F T'$$
$$T'\to *F T' | \varepsilon$$
$$F \to (E) | id$$

Step 3:

Consider the production $E'\to \varepsilon$. Since

$$FOLLOW(E') = \{ ), $ \}$$

Then
    
$$Table[E',)] = E'\to \varepsilon$$
$$Table[E',\$] = E'\to \varepsilon$$ 

## Result

For the following the non-left-recursive, left-factored expression grammar:

1. $E \to T E'$
2. $E'\to +T E'$
3. $E'\to \varepsilon$
4. $T \to F T'$
5. $T'\to *F T'$
6. $T'\to \varepsilon$
7. $F \to (E)$
8. $F \to id$ 


LL(1) parsing table is following:

N\T| id| + | * | ( | ) | $
---|---|---|---|---|---|---
E  | 1 |   |   | 1 |   |
E' |   | 2 |   |   | 3 | 3
T  | 4 |   |   | 4 |   |
T' |   | 6 | 5 |   | 6 | 6
F  | 8 |   |   | 7 |   |

## LL(1) Parser Driver

1. Push a $ on the stack.
2. Initialize the stack to the start symbol.
3. REPEAT WHILE stack is nonempty:
  
  - CASE top of the stack is:
    - terminal: IF input symbol matches terminal
      - THEN advance input and pop stack
      - ELSE error

    - nonterminal: Use nonterminal and current input  symbol to find correct production in table. 
      - Pop stack
      - Push right-hand side of production from table onto stack, last symbol first.


4. END REPEAT
5. If input is finished, THEN accept, ELSE error


## Example

See [LL(1) Parser](https://github.com/wildart/parsers/blob/master/ll1-parser.fsx)

# Bottom-up Parsers

## Bottom-up parsers

Given a right sentential form, $\alpha$:
- determine what substring of $\alpha$ is the right-hand side of the rule in the grammar that must be reduced to produce the previous sentential form in the right derivation

The most common bottom-up parsing algorithms are in the **LR** parser family.

## Handles

Definition: $\beta$ is the **handle** of the right sentential form $\gamma = \alpha \beta w$ if and only if $S => *_{rm} \alpha Aw =>_{rm} \alpha \beta w$

In this definition,
- $=>_{rm}$ specifies a rightmost derivation step, and
- $=> *_{rm}$ rm specifies zero or more rightmost derivation steps.


## Phrase

Definition: $\beta$ is the **phrase** of the right sentential form $\gamma$ if and only if $S \Rightarrow_* \gamma = \alpha_1 A \alpha_2 \Rightarrow_+ \alpha_1 \beta \alpha_2$

- $\Rightarrow_+$ means one or more derivation steps.

Definition: $\beta$ is the **simple phrase** of the right sentential form $\gamma$ if and only if $S \Rightarrow_* \gamma = \alpha_1 A \alpha_2 \Rightarrow \alpha_1 \beta \alpha_2$

- The definition of *phrase* uses **one or more steps**, while the definition of *simple phrase* uses **exactly one step**.

- A simple phrase is just a phrase that takes a single derivation step from root nonterminal node.


## Handles (cont.)

Intuition about handles (continued):
- The **handle** of a right sentential form is its leftmost **simple phrase**
- Given a parse tree, it is now easy to find the handle
- Parsing can be thought of as handle pruning

## Example

Consider the parse tree:

```
     E
  /  |   \
 /   |     T
 |   |  /  | \
 |   |  |  |  F
 |   |  |  |  |
 E   +  T  *  id
```

- There are three internal nodes, so three phrases
  - Each internal node is the root of a subtree, whose leaves are a phrase
    - E => E + T * id
    - T => T * id
    - F => id
- The simple phrases are a subset of the phrases
    - A simple phrase is always an RHS in the grammar
        - id
- The handle of any rightmost sentential form is its **leftmost simple phrase** - id.

## Shift-Reduce Algorithms

- Reduce is the action of replacing the handle on the top of the parse stack with its corresponding LHS
- Shift is the action of moving the next token to the top of the parse stack

## LR Parser

An LR configuration stores the state of an LR parser:

$$(S_0 X_1 S_1 X_2 S_2 \ldots X_m S_m, a_i a_{i+1} \ldots a_n \$)$$

LR parsers are table driven, where the table has two components, an **ACTION** table and a **GOTO** table
- The ACTION table specifies the action of the parser, given the parser state and the next token
    - Rows are state names; columns are terminals
- The GOTO table specifies which state to put on top of the parse stack after a reduction action is done
    - Rows are state names; columns are nonterminals

![](img/lr-parser.png)

## Parsing

Initial configuration:


$$(S_0 , a_1 a_2 \ldots a_n \$)$$

Parser actions:
- **Shift**
    - the next symbol of input is pushed onto the stack, along with the state symbol that is part of the Shift specification in the Action table
- **Reduce**:
    - remove the handle from the stack, along with its state symbols. Push the LHS of the rule. Push the state symbol from the GOTO table, using the state symbol just below the new LHS in the stack and the LHS of the new rule as the row and column into the GOTO table
- **Accept**
    - the parse is complete and no errors were found.
- **Error**:
    - the parser calls an error-handling routine.


## Example

This example of LR parsing uses the following small grammar with goal symbol `E`:

1. E → E * B
2. E → E + B
3. E → B
4. B → 0
5. B → 1

to parse the following input: **1 + 1**

The action table is indexed by a state of the parser and a terminal and contains three types of actions:

- shift, which is written as `sn` and indicates that the next state is `n`
- reduce, which is written as `rm` and indicates that a reduction with grammar rule `m` should be performed
- accept, which is written as `acc` and indicates that the parser accepts the string in the input stream.

## LR Parse Table

The two LR parsing tables for this grammar look as follows:

state | [   |     | action |     |  ]  | [go| to]|
------|-----|-----|:------:|-----|-----|----|----|
 |	*|	+|	0|	1|	$|	E|	B| 
0| | | s1| s2| | 3|	4|
1|	r4|	r4|r4|	r4|	r4| | |
2|	r5|	r5|	r5|	r5|	r5| | | 
3|	s5|	s6|   |   |acc| | | 
4|	r3|	r3|r3|r3|r3| | | 
5| | | s1| s2| | |	7|
6| | | s1| s2| | |	8|
7|	r1|	r1|	r1|	r1|	r1| | |	 	 
8|	r2|	r2|	r2|	r2|	r2| | | 

## Parsing steps

State | Input stream |Output stream | Stack | Next action
------|--------------|--------------|-------|-------------
0| 1+1\$ | | [0] | Shift 2
2|	+1\$ | |	[0,2] |	Reduce 5
4|	+1\$ | 5|	[0,4] |	Reduce 3
3|	+1\$ | 5,3|	[0,3] |	Shift 6
6|	 1\$ | 5,3|	[0,3,6]	| Shift 2
2|	  \$ | 5,3| [0,3,6,2] |	Reduce 5
8|	  \$	| 5,3,5| [0,3,6,8] |	Reduce 2
3|    \$ | 5,3,5,2|	[0,3] |	Accept

## Advantages of LR Parsers

- They will work for nearly all grammars that describe programming languages.
- They work on a larger class of grammars than other bottom-up algorithms, but are as efficient as any other bottom-up parser.
- They can detect syntax errors as soon as it is possible.
- The LR class of grammars is a superset of the  class parsable by LL parsers.

LR tables are genereated by tools, e.g. `yacc`

## Example

See [LR(0) Parser](parsers/lr0-parser.fsx)